# **Image Quality experiments**

In [1]:
import sys
import torch

from tqdm import tqdm
from pyiqa import create_metric
from torchvision import transforms
from torch.utils.data import DataLoader

sys.path.insert(0, "..\\Scripts")

c:\Users\tomma\OneDrive\Documenti\GitHub\FoodX-251_Classification\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from ImageDatastore import ImageDatastore

In [3]:
batch_size = 512
transforms = transforms.Compose(
    [
        transforms.Resize(
            (232, 232), interpolation=transforms.InterpolationMode.BILINEAR
        ),
    ]
)

val_small = ImageDatastore("val_set", transform=transforms)
val_deg = ImageDatastore("val_degradate", transform=transforms)

## **Quality Estimation**

In [4]:
# Initialize NIQE metric
niqe_metric = create_metric("niqe")

In [5]:
def calculate_niqe_scores(dataset):
    """
    Calculate NIQE scores for all images in the dataset
    """
    niqe_metric = create_metric('niqe')
    niqe_scores = []
    
    for idx in range(len(dataset)):
        image, _ = dataset[idx]
        # Add batch dimension and ensure image is on CPU
        image = image.unsqueeze(0).cpu()
        # Ensure values are exactly in [0, 1] range
        image = torch.clamp(image, 0.0, 1.0)
        # Double check the range
        assert image.min() >= 0.0 and image.max() <= 1.0, f"Image values out of range: min={image.min()}, max={image.max()}"
        niqe_score = float(niqe_metric(image))
        niqe_scores.append(niqe_score)
        
        # Print debug info for first few images
        if idx < 3:
            print(f"Image {idx} - min: {image.min():.6f}, max: {image.max():.6f}, shape: {image.shape}")
    
    return niqe_scores

In [6]:
# Initialize lists for NIQE scores
val_niqe = calculate_niqe_scores(val_small)

Image 0 - min: 0.000000, max: 1.000000, shape: torch.Size([1, 3, 232, 232])
Image 1 - min: 0.000000, max: 1.000000, shape: torch.Size([1, 3, 232, 232])
Image 2 - min: 0.000000, max: 0.997086, shape: torch.Size([1, 3, 232, 232])


In [7]:
deg_niqe = calculate_niqe_scores(val_deg)

Image 0 - min: 0.000000, max: 1.000000, shape: torch.Size([1, 3, 232, 232])
Image 1 - min: 0.000000, max: 1.000000, shape: torch.Size([1, 3, 232, 232])
Image 2 - min: 0.003801, max: 0.990090, shape: torch.Size([1, 3, 232, 232])


In [9]:
import pandas as pd

niqe_score = pd.DataFrame({'val_niqe': val_niqe, 'deg_niqe': deg_niqe})
niqe_score.describe()

,val_niqe,deg_niqe
count,11994.000000,11994.000000
mean,4.729814,6.304189
std,1.475678,2.510328
min,2.210318,2.245544
25%,3.810590,4.291391
50%,4.410388,5.682291
75%,5.260779,7.887836
max,21.560577,21.199060
